In [1]:
%load_ext autoreload
%autoreload 2

In [2]:

from main import *

100%|██████████| 986/986 [00:00<00:00, 6328.36it/s]


In [3]:
print(*config, sep='\n')

model_name
feature
max_len
min_freq_src
min_freq_trg
embedding_dim
hidden_size
num_layers
num_epochs
weight_decay
label_smoothing
dropout_emb
dropout_enc
dropout_dec
dropout_attention
learning_rate
gamma
patience
threshold
batch_size
use_tf
tf_from_epoch
tf_start
tf_decrease
src_vocab_size
trg_vocab_size


In [4]:
device='cuda'

In [5]:
vocab_src, vocab_trg, train_dataset, val_dataset

(<dataset.Vocab at 0x1df44902900>,
 <dataset.TranslationDataset at 0x1df448af4d0>)

In [6]:
from dataset import TrainDataLoader, TestDataLoader
from config import filenames, folders
from lstm3 import LSTM_3

In [7]:
unk_idx, pad_idx, bos_idx, eos_idx = 0, 1, 2, 3
device = 'cuda'

train_loader = TrainDataLoader(train_dataset, shuffle=True)
val_loader = TestDataLoader(val_dataset, shuffle=False)

weights_filename = folders['weights'] + 'LSTM_3-even-more-regularization-30.0m-15epoch.pt'

model = LSTM_3(config=config, weights_filename=weights_filename).to(device)
print(model)
print(model.count_parameters())

LSTM_3(
  (src_embedding): Embedding(24991, 120, padding_idx=1)
  (trg_embedding): Embedding(18710, 120, padding_idx=1)
  (emb_dropout): Dropout(p=0.2, inplace=False)
  (enc_dropout): Dropout(p=0.4, inplace=True)
  (dec_dropout): Dropout(p=0.4, inplace=True)
  (attention_dropout): Dropout(p=0.1, inplace=True)
  (encoder): LSTM(120, 360, num_layers=3, batch_first=True, dropout=0.4, bidirectional=True)
  (encoder_output_proj): Linear(in_features=720, out_features=360, bias=True)
  (decoder): LSTM(120, 360, num_layers=3, batch_first=True, dropout=0.4)
  (encoder_hidden_proj): ModuleList(
    (0-2): 3 x Linear(in_features=720, out_features=360, bias=True)
  )
  (encoder_cell_proj): ModuleList(
    (0-2): 3 x Linear(in_features=720, out_features=360, bias=True)
  )
  (fc): Linear(in_features=720, out_features=18710, bias=True)
)
30944870


In [8]:
import numpy as np
np.load('../weights/train.npy')

array([4.40982389, 3.8720361 , 3.65585768, 3.5329634 , 3.48615831,
       3.42260098, 3.40424037, 3.37793773, 3.34819674, 3.36216146,
       3.34456837, 3.34149617, 3.34147906, 3.35454029, 3.34359002])

In [9]:
from submission import get_bleu

In [35]:
from dataset import RawDataset
raw_dataset = RawDataset(filenames['test_trg'])

In [47]:
get_bleu(model, dataloader=val_loader, vocab_trg=vocab_trg, filenames=filenames, device=device, raw_dataset=raw_dataset, use_beam=True, beam_width=1)

100%|██████████| 4/4 [00:07<00:00,  1.82s/it]


precisions: [66.42670214 40.53904846 27.31464309 18.03492617]
precisions_cnt: [986. 986. 986. 986.]
avg bleu: 0.3004379914545143
penalties: 0.8852650415023444
bleu4: 27.10981744421907
sacrebleu ../data/val.de-en.en --tokenize none --width 2 -b -i ../submission/val_pred.en


27.89

In [12]:
from submission import make_submission
from dataset import SubmissionDataLoader, SubmissionDataset
raw_dataset = RawDataset(filenames['submission_src'])
submission_dataset = SubmissionDataset(filenames['submission_src'], vocab=vocab_src)
submission_loader = SubmissionDataLoader(submission_dataset)
make_submission(model, submission_loader=submission_loader, raw_dataset=raw_dataset, vocab_trg=vocab_trg, filenames=filenames)

100%|██████████| 24/24 [00:30<00:00,  1.27s/it]


In [13]:
val_dataset.device = 'cpu'

In [14]:
for a, b in val_dataset:
    print(a.device)
    break

cuda:0


In [15]:
get_bleu(model, dataloader=val_loader, vocab_trg=vocab_trg, filenames=filenames, device=device, raw_dataset=raw_dataset)

  0%|          | 0/4 [00:01<?, ?it/s]


KeyboardInterrupt: 

In [48]:
from submission import bleu

In [91]:
src, trg = val_dataset[20]

In [ ]:
" ".join(vocab_trg.decode(model.inference(src.unsqueeze(0)).squeeze(0)))
" ".join(vocab_trg.decode(trg))

'we were scared , but we wanted to go to school .'

In [90]:
bleu(vocab_trg.decode(model.inference(src.unsqueeze(0)).squeeze(0)), vocab_trg.decode(trg))

[0.45       0.21052632 0.05555556 0.02941176]
[45.         21.05263158  5.55555556  2.94117647]
0.8607079764250579


9.6

In [ ]:
model.demonstrate(val_loader, vocab_src, vocab_trg, device='cuda', examples=3, wait=5)

90
19 21 17 18
src:		 als ich <NUM> jahre alt war , wurde ich eines morgens von den klängen heller freude geweckt .
trg:		 when i was <NUM> , i remember waking up one morning to the sound of joy in my house .
pred:		 when i was <NUM> years old , i was told one morning by the bright ,
0.24
pred-beam:	 when i was <NUM> years old , i was told by the sounds of the noises .
0.17

17 15 14 16
src:		 mein vater hörte sich auf seinem kleinen , grauen radio die der bbc an .
trg:		 my father was listening to bbc news on his small , gray radio .
pred:		 my father was clinging to his little , gray radio
0.07
pred-beam:	 my dad listened to his little , radio , the little radio waves .
0

20 21 18 22
src:		 er sah sehr glücklich aus , was damals ziemlich ungewöhnlich war , da ihn die nachrichten meistens .
trg:		 there was a big smile on his face which was unusual then , because the news mostly depressed him .
pred:		 he looked very happy , which was quite unusual , because it 's the news mostly .
0

In [ ]:
from submission import bleu, get_precisions
ref = "there was a big smile on his face which was unusual then , because the news mostly depressed him ."
pred = "he looked very happy , which was quite unusual , because it 's the news mostly ."
bleu(ref.split(), pred.split())

NameError: name 'get_precisions' is not defined

In [ ]:
raw_submission = RawDataset(filenames['submission_src'])

In [ ]:
from submission import make_submission
from dataset import SubmissionDataset, SubmissionDataLoader
submission_dataset = SubmissionDataset(filenames['submission_src'], vocab_src, device=device)
ldr = SubmissionDataLoader(submission_dataset)
# make_submission(model, ldr, vocab_trg, filenames, device=device, raw_dataset=raw_submission)

In [ ]:
src, trg = val_dataset[0]

In [ ]:
from dataset import TestDataLoader
ldr = TestDataLoader(val_dataset, batch_size=2)
src, trg = ldr[0]
src.size(1)

In [ ]:
import torch
from torch.nn.functional import softmax
trg_words = ['<BOS>', 'i', 'was', '<NUM>', 'years', 'i', 'was']
i = len(trg_words) - 1
trg_pred = torch.tensor([vocab_trg.encode_word(word) for word in trg_words], dtype=torch.long).unsqueeze(0)
predictions = model.inference(src, device='cpu')
print(vocab_trg.decode(trg_pred.squeeze(0).numpy()))
for idx in predictions[0]:
    print(vocab_trg.decode_idx(idx.item()), end=' ')